# Import Package

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

#text package
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Import Datasets

In [38]:
df = pd.read_json('data/crawling-data-twitter-psbb.json',lines=True)
df.head()

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status
0,2020-07-07 05:29:34+00:00,1280373372623282176,1280373372623282176,RT @luckytribeid: ✨ Give Away Voucher Go-Pay R...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,209,0,False,False,in,NaN,NaN,NaN,NaN,NaN
1,2020-07-07 05:28:21+00:00,1280373065952591872,1280373065952591872,#surabaya\n#SurabayaBermasker\n\nTekan Penyeba...,True,"{'hashtags': [{'text': 'surabaya', 'indices': ...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,0,0,False,False,in,0.0,NaN,NaN,NaN,NaN
2,2020-07-07 05:26:03+00:00,1280372485137944576,1280372485137944576,"Penularan Covid-19 Naik, Epidemiolog: Konsekue...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",NaN,NaN,...,0,4,False,False,in,0.0,NaN,NaN,NaN,NaN
3,2020-07-07 05:25:08+00:00,1280372255214592003,1280372255214592000,@AdiNotonegoro_ @halleluhellyeah Terpantau nge...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.280110e+18,1.280110e+18,...,0,0,False,False,in,0.0,"{'media': [{'id': 1280372248998625280, 'id_str...",NaN,NaN,NaN
4,2020-07-07 05:23:14+00:00,1280371776690614273,1280371776690614272,@dpu_UUT Wkwkwk psbb ta?,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.280371e+18,1.280371e+18,...,0,0,False,False,in,NaN,NaN,NaN,NaN,NaN


In [39]:
columns = ['created_at', 'text']
data = pd.DataFrame(df, columns=columns).copy()
data.head(10)

,created_at,text
0,2020-07-07 05:29:34+00:00,RT @luckytribeid: ✨ Give Away Voucher Go-Pay R...
1,2020-07-07 05:28:21+00:00,#surabaya\n#SurabayaBermasker\n\nTekan Penyeba...
2,2020-07-07 05:26:03+00:00,"Penularan Covid-19 Naik, Epidemiolog: Konsekue..."
3,2020-07-07 05:25:08+00:00,@AdiNotonegoro_ @halleluhellyeah Terpantau nge...
4,2020-07-07 05:23:14+00:00,@dpu_UUT Wkwkwk psbb ta?
5,2020-07-07 05:21:26+00:00,"*_Bhabinkamtibmas Kel, Sindangrasa Aiptu Maman..."
6,2020-07-07 05:21:03+00:00,"Longgarkan PSBB, Kini Ojol Bisa Angkut Penumpa..."
7,2020-07-07 05:20:36+00:00,RT @detikcom: Dinas Parekraf DKI Jakarta mengi...
8,2020-07-07 05:18:23+00:00,"Cocok buat kaliam yang introvert, haha. Atau m..."
9,2020-07-07 05:17:36+00:00,Hidup di masa PSBB kemarin dan sekarang udah b...
